In [1]:
import numpy as np
import pandas as pd
import networkx as nx

from time import sleep
from tqdm import tqdm

In [2]:
foldername = '0084'
n_inds = 30
scale = "30"

angle_thresh = 30
time_thresh = 5
thresh_folder = 'angle' + str(angle_thresh) + '_time' + str(time_thresh)

In [3]:
ind = pd.read_csv('/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/DirectionalCorrelation/Data/Output/golden_shiners/' + str(n_inds) + '_fish/' + foldername + '/' + thresh_folder + '/individual_' + scale + '.csv')
ind.drop(['Unnamed: 0'], axis=1, inplace=True)

pair = pd.read_csv('/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/DirectionalCorrelation/Data/Output/golden_shiners/' + str(n_inds) + '_fish/' + foldername + '/' + thresh_folder + '/pairwise_' + scale + '.csv')
pair.drop(['Unnamed: 0'], axis=1, inplace=True)
pair.head()

,f_id,n_id,dist,ang_area,speed_diff,acc_diff,frame,tau,cc,ang_pos
0,0,1,315.757818,0.000000,0.551799,0.005935,450,-128,0.0,0.961767
1,0,2,111.924099,0.367566,-0.082841,-0.009714,450,0,0.0,1.648288
2,0,3,154.803262,0.075398,0.156155,0.001394,450,-12,0.0,1.113337
3,0,4,290.640091,0.000000,0.435032,0.013989,450,-150,0.0,0.881489
4,0,5,300.684983,0.000000,0.687985,0.032424,450,-92,0.0,0.624697


In [4]:
assert(np.min(ind['frame']) == np.min(pair['frame'])) 
assert(np.max(ind['frame']) == np.max(pair['frame']))

In [5]:
df_ind = pair.loc[:,['f_id', 'frame']].drop_duplicates()
df_ind['reach'] = 0

for fr in tqdm(np.unique(pair['frame'])):
    tmp = pair[(pair['frame'] == fr) & (pair['cc'] == 1)].reset_index()
    
    G = nx.DiGraph()
    G.add_nodes_from(np.unique(pair['f_id']))
    
    list_a = list(tmp.loc[tmp['cc'] == 1, 'n_id'].values) 
    list_b = list(tmp.loc[tmp['cc'] == 1, 'f_id'].values)
    
    if len(list_a) != 0:
        G.add_edges_from(list(zip(list_a, list_b)))

        for idx in np.unique(pair['n_id']):
            df_ind.loc[(pair['frame'] == fr) & (pair['f_id'] == idx), 'reach'] = nx.local_reaching_centrality(G, idx)
        
    sleep(0.001)
    
df = pd.merge(ind,df_ind)

100%|██████████████████████████████████| 23692/23692 [21:38:43<00:00,  3.29s/it]


In [6]:
df.to_csv('/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/DirectionalCorrelation/Data/Output/golden_shiners/' + str(n_inds) + '_fish/' + foldername + '/' + thresh_folder + '/individual_' + scale + '.csv', mode='w')